In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')


In [3]:
%%px
# Import the libraries

import pyfftw as ft 
import numpy as np
from mpi4py import MPI
import math
import sys
import os
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

from FFT3Dfield import FFT3Dfield
from IFFT3Dfield import IFFT3Dfield
from EnergySpectrum import EnergySpectrum

In [4]:
%%px

import os
import sys
import requests
from tempfile import NamedTemporaryFile
import numpy as np
import h5py

In [5]:
%%px

comm = MPI.COMM_WORLD
my_id = comm.Get_rank()
nproc = comm.Get_size()
if(my_id==0):
    print("n_proc = "+str(nproc))
    print("my_id = "+str(my_id))

[stdout:4] 
n_proc = 8
my_id = 0


In [6]:
%%px

def get_cutout(
        x0, y0, z0, t0, xl, yl, zl, tl,
        filename, data_set, data_type,
        auth_token, base_website):
    url = ('http://' + base_website + '/jhtdb/getcutout/'
         + auth_token + '/'
         + data_set + '/' + data_type + '/'
         + '{0},{1}/'.format(t0, tl)
         + '{0},{1}/'.format(x0, xl)
         + '{0},{1}/'.format(y0, yl)
         + '{0},{1}/'.format(z0, zl))
    print(url)
    if data_type in ['u', 'b', 'a']:
        ncomponents = 3
    elif data_type in ['p']:
        ncomponents = 1
    elif data_type in ['ub']:
        ncomponents = 6
    try:
        #print('Retrieving h5 file, size {0} MB = {1} MiB.'.format(
        #    xl*yl*zl*ncomponents * 4. / 10**6,
        #    xl*yl*zl*ncomponents * 4. / 2**20))
        u = requests.get(url)
    except:
        print("Download failed")
    tempfile = NamedTemporaryFile()
    
    tempfile.write(u.content)
    tempfile.seek(0)
    h5file = h5py.File(tempfile.name, 'r')
    for i in iter(h5file.keys()):
        if (i[0] != "_"): #Skip metdata fields
            tmpfieldz = h5file[i][:,:,:,0] 
            tmpfieldy = h5file[i][:,:,:,1] 
            tmpfieldx = h5file[i][:,:,:,2] 
            tmpfield = (tmpfieldx.copy(),tmpfieldy.copy(),tmpfieldz.copy())   
            return tmpfield

In [12]:
%%px 
x0 = 0
y0 = 0
z0 = 0
xl = 32
yl = 512
zl = 512
t0 = 0
tl = 1
ts = 0 #timestep
filename = 'temp'
data_set = 'isotropic1024coarse'
data_type = 'u'
auth_token = 'com.gmail.jhelsas-b854269a'
base_website = 'dsp033.pha.jhu.edu'

#print ("Data is: ")
#print (data)

In [13]:
%%px

comm.Barrier(); t1=MPI.Wtime()

for k in range(4):
    x0 = my_id*128+k*32
    data = get_cutout(z0, y0, x0, ts, zl, yl, xl, 1,
                      filename, data_set, data_type,
                      auth_token, base_website)

comm.Barrier(); t2=MPI.Wtime()

if(my_id==0):
    print("Finished loading")
    sys.stdout.write('Load velocity field cost: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] 
http://dsp033.pha.jhu.edu/jhtdb/getcutout/com.gmail.jhelsas-b854269a/isotropic1024coarse/u/0,1/0,512/0,512/896,32/
http://dsp033.pha.jhu.edu/jhtdb/getcutout/com.gmail.jhelsas-b854269a/isotropic1024coarse/u/0,1/0,512/0,512/928,32/
http://dsp033.pha.jhu.edu/jhtdb/getcutout/com.gmail.jhelsas-b854269a/isotropic1024coarse/u/0,1/0,512/0,512/960,32/
http://dsp033.pha.jhu.edu/jhtdb/getcutout/com.gmail.jhelsas-b854269a/isotropic1024coarse/u/0,1/0,512/0,512/992,32/
[stdout:1] 
http://dsp033.pha.jhu.edu/jhtdb/getcutout/com.gmail.jhelsas-b854269a/isotropic1024coarse/u/0,1/0,512/0,512/256,32/
http://dsp033.pha.jhu.edu/jhtdb/getcutout/com.gmail.jhelsas-b854269a/isotropic1024coarse/u/0,1/0,512/0,512/288,32/
http://dsp033.pha.jhu.edu/jhtdb/getcutout/com.gmail.jhelsas-b854269a/isotropic1024coarse/u/0,1/0,512/0,512/320,32/
http://dsp033.pha.jhu.edu/jhtdb/getcutout/com.gmail.jhelsas-b854269a/isotropic1024coarse/u/0,1/0,512/0,512/352,32/
[stdout:2] 
http://dsp033.pha.jhu.edu/jhtdb/getcutout/co

In [7]:
%%px
print(my_id)
datax = data[0]
datay = data[1]
dataz = data[2]
print(datax)

[stdout:0] 
1
[[[-0.13649869 -0.15233463 -0.15298946 -0.15078452]
  [-0.10481487 -0.12159604 -0.11943598 -0.10821148]
  [-0.06764074 -0.07264471 -0.07613516 -0.07476459]
  [-0.07136229 -0.05779634 -0.0555416  -0.05205235]]

 [[-0.16289109 -0.16532555 -0.15954202 -0.15866947]
  [-0.15231247 -0.15401916 -0.14995076 -0.13893498]
  [-0.12326394 -0.1119568  -0.10603371 -0.09855276]
  [-0.12838015 -0.09966379 -0.08377291 -0.06739188]]

 [[-0.16909519 -0.16075216 -0.16936672 -0.17517063]
  [-0.18158965 -0.17102408 -0.17811435 -0.17615464]
  [-0.18420531 -0.15172142 -0.13262641 -0.12270341]
  [-0.17992005 -0.1434316  -0.11536966 -0.09386986]]

 [[-0.15319322 -0.16124508 -0.17818725 -0.20483413]
  [-0.16339351 -0.17605615 -0.19135424 -0.20315926]
  [-0.18796083 -0.17470494 -0.17039719 -0.16881824]
  [-0.18801835 -0.1690723  -0.16104031 -0.15374562]]]
[stdout:1] 
5
[[[-0.13649869 -0.15233463 -0.15298946 -0.15078452]
  [-0.10481487 -0.12159604 -0.11943598 -0.10821148]
  [-0.06764074 -0.07264471 -